In [1]:
from torch.utils.data import DataLoader, Dataset

from lightning import LightningDataModule

from src.data.components.data import FuturePredictionDataset
from omegaconf import DictConfig

In [2]:
from omegaconf import OmegaConf

common = OmegaConf.create({
    'semantic_segmentation': {
        'weights': [1.0, 2.0],
        'use_top_k': True,
        'top_k_ratio': 0.25,
    },
    'receptive_field': 3,
    'future_discount': 0.95,
    'ignore_index': 255,
    'lift': {
        'x_bound': [-50.0, 50.0, 0.5],
        'y_bound': [-50.0, 50.0, 0.5],
        'z_bound': [-10.0, 10.0, 20.0],
        'd_bound': [2.0, 50.0, 1.0],
    },
    'image': {
        'final_dim': [224, 480],
        'resize_scale': 0.3,
        'top_crop': 46,
        'original_height': 900,
        'original_width': 1600,
        'names': ['CAM_FRONT_LEFT', 'CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_BACK_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT'],
    },
})



In [3]:
import os

data_root = '/home/slabban/machine_learning/projects/bev-hydra/data/nuscenes'
version = 'trainval'
batch_size = 1
filter_invisible_vehicles = True
num_workers = 4
pin_memory = True


In [4]:
data_train = FuturePredictionDataset(
            data_root=data_root, is_train=True, version=version, batch_size=batch_size, filter_invisible_vehicles=filter_invisible_vehicles, common=common
        )

In [5]:
data_loader = DataLoader(
            dataset=data_train,
            batch_size=batch_size,
            num_workers=num_workers,
            pin_memory=pin_memory,
            shuffle=True, drop_last= True
        )

In [6]:
sample = next(iter(data_loader))

In [9]:
print(sample.keys())

dict_keys(['image', 'intrinsics', 'extrinsics', 'segmentation', 'instance', 'centerness', 'offset', 'flow', 'future_egomotion', 'sample_token', 'z_position', 'attribute'])


In [10]:
for future_ego_pose in sample['future_egomotion']:
    print(future_ego_pose)

tensor([[-4.1266e+00,  4.2961e-02,  2.5883e-02,  1.9745e-03, -2.6767e-03,
         -4.7825e-03],
        [-6.1031e+00,  7.6385e-02,  2.7347e-02, -1.0644e-03, -1.7784e-03,
         -2.1298e-03],
        [-5.0339e+00,  2.9167e-02,  1.5949e-02, -1.3499e-03, -1.3432e-03,
         -4.2299e-04]])


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_all_camera_images_at_timestep(sample, timestep=0):
    """
    Visualizes all camera images from a specific timestep of a given sample in the dataset.
    
    Parameters:
    - dataset: instance of FuturePredictionDataset or similar.
    - sample_index: Index of the sample to visualize.
    - timestep: Timestep of the images to visualize.
    """
    # Fetch the sample data
    images_tensor = sample['image']  # Assuming shape is (T, N, 3, H, W)
    
    # Select all images at the given timestep
    timestep_images = images_tensor[timestep]  # This selects all N camera images at the given timestep
    
    # Create a subplot for each camera image
    num_cameras = timestep_images.shape[0]
    fig, axs = plt.subplots(1, num_cameras, figsize=(15, 10))
    
    for i, img in enumerate(timestep_images):
        # img is of shape (3, H, W), need to permute for plotting
        axs[i].imshow(img.permute(1, 2, 0).numpy())
        axs[i].set_title(f'Camera {i+1}')
        axs[i].axis('off')
    
    plt.show()

# Example usage
visualize_all_camera_images_at_timestep(dataset, sample_index=0, timestep=0)  # Adjust sample_index as needed